In [1]:
import qrcode
# import zxing
from qrtools import qrtools
from os import path
import compress.api as compress
from tqdm.notebook import tqdm
from PIL import Image
from pyzbar.pyzbar import decode

In [7]:
def bin_to_qr(data, chunk_size=100,folder_path = None ,filename_prefix="qr_code", box_size=1, border=10):
    compressed_data = compress_bytes(data)
    hex_data = compressed_data.hex()    
    chunks = [hex_data[i:i+chunk_size] for i in range(0, len(hex_data), chunk_size)]
    total_chunks = len(chunks)

    for i, chunk in enumerate(tqdm(chunks), 1):
        qr = qrcode.QRCode(version=40, error_correction=qrcode.constants.ERROR_CORRECT_L, box_size=box_size, border=border)
        qr.add_data(f"{i}/{total_chunks}:{chunk}")
        qr.make(fit=False)
        print(f"Generating QR code {i}", end="")
        img = qr.make_image(fill_color="black", back_color="white")
        out_path = path.join(folder_path, f"{filename_prefix}_{i}.png")
        img.save(out_path)
        print(f"[OK]")

    print(f"Generated {total_chunks} QR codes.")
    print(f"Each QR code except the last one will contain {chunk_size} bytes of data.")
    print(f"The last QR code will contain the remaining bytes of data ({len(chunks[-1]  )} bytes).")

def qr_to_bin(folder_path = None,filename_prefix="qr_code"):
    chunks = {}
    i = 1
    # reader = zxing.BarCodeReader()
    # fallback_reader = qrtools.QR
    

    while True:
        try:
            filename = path.join(folder_path, f"{filename_prefix}_{i}.png")
            print(f"Decoding QR code {i}...")
            # print(filename)
            # barcode = reader.decode(filename)
            decoded = decode(Image.open(filename))[0].data.decode("utf-8")


            
            print(f"decoded...", end="")
            # if barcode and barcode.parsed:
            #     decoded = barcode.parsed
            # else:
            #     print(f"Could not decode QR code {i}: {barcode.raw} with zxing, trying fallback reader...")
                
            # split the decoded string into chunk_info and chunk_data
            chunk_info, chunk_data = decoded.split(':', 1)
            chunk_num, total_chunks = map(int, chunk_info.split('/'))
            chunks[chunk_num] = chunk_data
            # print(chunk_data)
            print(f"binary data extracted [OK]")
            if chunk_num == total_chunks:
                break
                # print(fallback_reader.data)
            if not chunks[i]:
                print(f"Error decoding QR code {i}")
                exit(-1)
            print(f"binary data extracted [OK]")

            i += 1
        except FileNotFoundError:
            break
        except Exception as e:
            print(f"Error decoding QR code {i}: {e}")
            raise e
            exit(-1)

    if not chunks:
        print("No QR codes found.")
        return None

    hex_data_compressed = ''.join(chunks[i] for i in range(1, len(chunks) + 1))
    bytes_compressed = bytes.fromhex(hex_data_compressed)
    return bytes_compressed

# Compressing as much as possible

def compress_bytes(data):
    data_compressed = compress.compress(
    algo=compress.Algorithm.gzip,
    data=data,
    kwargs={"compresslevel": 9})
    return data_compressed

def decompress_bytes(data):
    # compressor = Compressor()
    # compressor.use_gzip()
    return compress.decompress(algo=compress.Algorithm.gzip,data=data, kwargs={"compresslevel": 9})

In [8]:
with open("test_file.JPG", "rb") as f:
    data = f.read()

bin_to_qr(data, folder_path='out_path/'  ,chunk_size=2048, box_size=1, border=5)

  0%|          | 0/4232 [00:00<?, ?it/s]

Generating QR code 1[OK]
Generating QR code 2[OK]
Generating QR code 3[OK]
Generating QR code 4[OK]
Generating QR code 5[OK]
Generating QR code 6[OK]
Generating QR code 7[OK]
Generating QR code 8[OK]
Generating QR code 9[OK]
Generating QR code 10[OK]
Generating QR code 11[OK]
Generating QR code 12[OK]
Generating QR code 13[OK]
Generating QR code 14[OK]
Generating QR code 15[OK]
Generating QR code 16[OK]
Generating QR code 17[OK]
Generating QR code 18[OK]
Generating QR code 19[OK]
Generating QR code 20[OK]
Generating QR code 21[OK]
Generating QR code 22[OK]
Generating QR code 23[OK]
Generating QR code 24[OK]
Generating QR code 25[OK]
Generating QR code 26[OK]
Generating QR code 27[OK]
Generating QR code 28[OK]
Generating QR code 29[OK]
Generating QR code 30[OK]
Generating QR code 31[OK]
Generating QR code 32[OK]
Generating QR code 33[OK]
Generating QR code 34[OK]
Generating QR code 35[OK]
Generating QR code 36[OK]
Generating QR code 37[OK]
Generating QR code 38[OK]
Generating QR code 39

ValueError: glog(0)

In [ ]:
test_file_compressed = qr_to_bin(folder_path='out_path/')

In [ ]:
test_file = decompress_bytes(test_file_compressed)
with open( "test_file_out.JPG", "wb") as f:
     f.write(test_file)